SMA (dataframe,str,int)   ;MACD (dataframe,str,(1,3)list[int])     ;  MAX (dataframe,str,int)    ; MIN (dataframe,str,int)

STD (dataframe,str,int)    ; CCI (dataframe,str,int)      ;RSI (dataframe,str,int)     ; STOCASTIC (dataframe,str,(1,2)list[int]) 

In [11]:
import datetime as dt
import numpy as np
import pandas as pd
import warnings
import math
import matplotlib.pyplot as plt
import scipy.optimize
from scipy import stats
from scipy.optimize import OptimizeWarning
from sklearn.linear_model import LinearRegression
from matplotlib.dates import date2num


In [2]:
def SMA(df,column,period):
    df[column+'Sma'+str(period)]=(df[column]).rolling(window=period).mean()
    return df

In [3]:
def MACD (df,column,period):
    exp1 = (df[column]).ewm(span=period[0], adjust=False).mean()
    exp2 = (df[column]).ewm(span=period[1], adjust=False).mean()
    macd = exp1-exp2
    exp3 = macd.ewm(span=period[2], adjust=False).mean()
    df[column+'Macd'+str(period[0])+'_'+str(period[0])]=macd*100
    df[column+'MacdSignal'+str(period[2])]=exp3*100
    return df

In [4]:
def MAX(df,column,period):
    df[column+'Max'+str(period)]=(df[column]).rolling(window=period).max()
    return df

In [5]:
def MIN(df,column,period):
    df[column+'Min'+str(period)]=(df[column]).rolling(window=period).min()
    return df

In [6]:
def STD(df,column,period):
    df[column+'Std'+str(period)]=(df[column]).rolling(window=period).std()*100
    return df

In [7]:
def CCI(df,column,period):
    df[column+'Cci'+str(period)]=((df[column]) - (df[column]).rolling(window=period).mean()) / (1.5 * (df[column]).rolling(window=period).std())
    return df

In [8]:
def RSI (df,column,period):
    df[column+'Gain']=((df[column])-(df[column]).shift(periods=1)).clip(lower=0)
    df[column+'Loss']=((df[column]).shift(periods=1)-(df[column])).clip(lower=0)
    RSAsk=(df[column+'Gain']).rolling(window=period).mean()/(df[column+'Loss']).rolling(window=period).mean()
    df[column+'Rsi'+str(period)]=100-100/(1+RSAsk)

In [9]:
def STOCASTIC(df,period):
    df['%K'+str(period[0])]=(df.Close-df.Low.rolling(window=period[0]).min())/(df.High.rolling(window=period[0]).max()-df.Low.rolling(window=period[0]).min())
    df['%D'+str(period[1])]=(df['%K'+str(period[0])]).rolling(window=period[1]).mean()
    return df

In [15]:
def HACANDLE(df):
    df['HaClose']=(df.Open+df.High+df.Low+df.Close)/4
    Open=df.HaClose[0]
    df['HaOpen']=df['HaClose']
    for i in range(1,len(df.HaOpen)):
        df.HaOpen[i]=(df.HaOpen[i-1]+df.HaClose[i-1])/2
    df['HaHigh']=df.max(axis=1)
    df['HaLow']=df.min(axis=1)

In [17]:
def fu(x,a0,a1,b1,w):
    return(a0+a1*np.cos(w*x)+b1*np.sin(w*x))

In [18]:
def si(x,a0,b1,w):
    return(a0+b1*np.sin(w*x))

In [ ]:
def FIT(df,fun,coff):
    coefficients=scipy.optimize.curve_fit(fun,np.arange(0,len(df)),df)
    return(coefficients[coff])

In [16]:
def FURIERFIT(df,period):
    detrended=df.Close-df.Close.shift(periodes=1)
    df['SinFitA0']=detrended.rolling(window=period).apply(FIT,kwargs={fun:si,coff:0})
    
    
                
              
    